<a href="https://colab.research.google.com/github/iamsoroush/mnist_inception_finetune/blob/master/mnist_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a id="1"></a> <br>
# Import Required Libraries

As the first step, we need to import needed libraries

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Input, UpSampling3D
from keras.models import Model
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.utils import plot_model
from keras.utils.np_utils import to_categorical
from keras.applications.inception_v3 import InceptionV3

from keras.datasets import mnist

# Load and Prepare Data

In [24]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('X train: ', x_train.shape)
print('y train: ', y_train.shape)
print('X test: ', x_test.shape)
print('y test: ', y_test.shape)

x_train = (x_train / 255).astype('float32')
x_test = (x_test / 255).astype('float32')

X train:  (60000, 28, 28)
y train:  (60000,)
X test:  (10000, 28, 28)
y test:  (10000,)


The Input in in th shape 1 * 784 arrays. We convert them to an image of 28 * 28 pixel to be able to use convolutional layers.


In [25]:
#expand 1 more dimention as 1 for colour channel gray
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
print(x_train.shape)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


We have a number for each image but we need an array.
2 has to be converted to [0, 0, 1, 0, 0 ,0 ,0 ,0 ,0 ,0] 

In [26]:
y_train = to_categorical(y_train)
num_classes = y_train.shape[1]
y_train.shape

(60000, 10)

Split training set to train and validation sets:

In [0]:
# Set the random seed
random_seed = 2
# Split the train and the validation set for the fitting
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=random_seed)

<a id="9"></a> <br>
# Define the Model

In [28]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)


input_inception = Input(shape=(28, 28, 1), dtype='float32', name='inception_input')

# create the base pre-trained model
x = UpSampling3D(size=(3, 3, 3), data_format="channels_last")(input_inception)
# x = Conv2D(filters=3, kernel_size=1, padding="same", activation='relu', data_format='channels_last')(x)

x = base_model(x)

# add a global spatial average pooling layer
# x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
# and a logistic layer
predictions = Dense(10, activation='softmax')(x)

# this is the model we will train
inceptionv3_model = Model(inputs=input_inception, outputs=predictions)

inceptionv3_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_input (InputLayer) (None, 28, 28, 1)         0         
_________________________________________________________________
up_sampling3d_3 (UpSampling3 (None, 84, 84, 3)         0         
_________________________________________________________________
inception_v3 (Model)         multiple                  21802784  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                10250     
Total para

# Freeze the First 2 Blocks

In [29]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_3
1 conv2d_189
2 batch_normalization_189
3 activation_189
4 conv2d_190
5 batch_normalization_190
6 activation_190
7 conv2d_191
8 batch_normalization_191
9 activation_191
10 max_pooling2d_9
11 conv2d_192
12 batch_normalization_192
13 activation_192
14 conv2d_193
15 batch_normalization_193
16 activation_193
17 max_pooling2d_10
18 conv2d_197
19 batch_normalization_197
20 activation_197
21 conv2d_195
22 conv2d_198
23 batch_normalization_195
24 batch_normalization_198
25 activation_195
26 activation_198
27 average_pooling2d_19
28 conv2d_194
29 conv2d_196
30 conv2d_199
31 conv2d_200
32 batch_normalization_194
33 batch_normalization_196
34 batch_normalization_199
35 batch_normalization_200
36 activation_194
37 activation_196
38 activation_199
39 activation_200
40 mixed0
41 conv2d_204
42 batch_normalization_204
43 activation_204
44 conv2d_202
45 conv2d_205
46 batch_normalization_202
47 batch_normalization_205
48 activation_202
49 activation_205
50 average_pooling2d_20
51 conv2d_201
52 

In [30]:
for layer in base_model.layers[:64]:
   layer.trainable = False
    
inceptionv3_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_input (InputLayer) (None, 28, 28, 1)         0         
_________________________________________________________________
up_sampling3d_3 (UpSampling3 (None, 84, 84, 3)         0         
_________________________________________________________________
inception_v3 (Model)         multiple                  21802784  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                10250     
Total para

# Compile and Fit the Model

In [0]:
# Define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [0]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.005, 
                                            min_lr=0.00001)

In [0]:
inceptionv3_model.compile(optimizer=optimizer , loss="categorical_crossentropy", metrics=["accuracy"])

In [34]:
inceptionv3_history = inceptionv3_model.fit(x_train, y_train, epochs=20, batch_size=64,
                                            validation_data=(x_val, y_val),
                                            shuffle=True,
                                            callbacks=[learning_rate_reduction])

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 224s 5ms/step - loss: 0.4239 - acc: 0.9119 - val_loss: 1.0341 - val_acc: 0.9090
Epoch 2/20
48000/48000 [==============================] - 199s 4ms/step - loss: 0.2916 - acc: 0.9680 - val_loss: 1.1052 - val_acc: 0.9209
Epoch 3/20
48000/48000 [==============================] - 199s 4ms/step - loss: 0.2489 - acc: 0.9732 - val_loss: 1.7698 - val_acc: 0.8823
Epoch 4/20
48000/48000 [==============================] - 193s 4ms/step - loss: 0.3403 - acc: 0.9701 - val_loss: 0.3789 - val_acc: 0.9671
Epoch 5/20
48000/48000 [==============================] - 192s 4ms/step - loss: 0.2848 - acc: 0.9746 - val_loss: 0.2784 - val_acc: 0.9764
Epoch 6/20
48000/48000 [==============================] - 192s 4ms/step - loss: 0.2935 - acc: 0.9763 - val_loss: 0.2274 - val_acc: 0.9796
Epoch 7/20
48000/48000 [==============================] - 194s 4ms/step - loss: 0.0910 - acc: 0.9889 - val_loss: 0.1075 - 